In [ ]:
import re
import math

DIV_COEFF = 2.5
Y_OFFSET = 50

def load_content(filename):
    with open(filename, "r") as file:
        content = file.read()

    paths = re.findall(r"\<path(?:.*?)d=\"(.*)\"(?:.*?)/>", content)
    road_path = paths[0]
    starting_lines = paths[1:]
    return road_path, starting_lines
    
def normalize_road_path(road_path):
    res = []
    state = 0
    for x in road_path.split(" "):
        try:
            x = float(x) // DIV_COEFF

            if state % 2:
                x += Y_OFFSET
            state += 1
        except ValueError:
            pass
        res += [str(x)]
    return " ".join(res)


def compute_starting_positions(starting_lines):
    res = []
    for line in starting_lines:
        line = line.strip()
        _, x1, y1, _, x2, y2 = line.split(" ")

        def fn(x):
            return float(x) / DIV_COEFF

        x1, y1, x2, y2 = list(map(fn, (x1, y1, x2, y2)))
        y1 += Y_OFFSET
        y2 += Y_OFFSET

        if y1 == y2:
            if x1 > x2:
                angle = 3 * math.pi / 2
            else:
                angle = math.pi / 2
        elif x1 == x2:
            if y1 > y2:
                angle = 0
            else:
                angle = math.pi
        else:
            invert = False
            if y2 > y1:
                invert = True
            if x1 > x2:
                invert = not invert
            if invert:
                angle = math.atan(abs((y1 - y2)/(x2 - x1)))
            else:
                angle = math.atan(abs((x2 - x1)/(y1 - y2)))
            if x2 < x1:
                angle += math.pi
                if y2 < y1:
                    angle += math.pi / 2
            elif y2 > y1:
                angle += math.pi / 2

        res += [[x1, y1, angle]]
    return res


def to_string(road_path, starting_positions):
    res = '{\n  "path": "' + road_path + '",\n  "starting_positions": ['
    for x in starting_positions:
      res += "\n    " + str(x) + ","
    res = res[:-1]
    res += "\n  ]\n}"
    print(res)

    


In [ ]:
filename = "C:\\Users\\v-remip\\Documents\\racing\\tr7.svg"

road_path, starting_lines = load_content(filename)

to_string(
    road_path=normalize_road_path(road_path),
    starting_positions=compute_starting_positions(starting_lines),
)